# PGCS Number Scraper

This notebook is designed to obtain all the neccesary information to complete create an image scraper.

## Get HTML

Get the HTML from the PCGS website.

### Imports

Import `cloudscraper` for scraping. Import `pandas` for data manipulation.

In [ ]:
from bs4 import BeautifulSoup
import cloudscraper
import pandas as pd
import numpy as np

### Headers and scraper

In [ ]:
# Establish connectection and pull HTML
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:70.0) Gecko/20100101 Firefox/70.0',
         'referer': 'https://www.google.com/',
         'Upgrade-Insecure-Requests': '1'}
url = 'https://www.pcgs.com/prices/detail/mercury-dime/703/most-active/ms?pn=1&ps=-1'
scraper = cloudscraper.create_scraper()
status = scraper.get(url)
html = status.text 

print('Status code: ' + str(status.status_code))

### Get table from PCGS

For now, we don't need the description as we originally thought. Might delete the manuel indexings later.

In [ ]:
# Creates and cleans the table
df = pd.read_html(html)
df = pd.DataFrame(df[0],dtype=object)
df = df.drop([0,1])
df = df.rename(columns=df.iloc[0])
df = df[['PCGS #','Description']].copy()
df = df.rename(columns={'PCGS #': 'id', 'Description': 'description'})
df = df.dropna()
df = df.reset_index(drop=True)

# Manual indexing check if error occurs, pcgs might change something
df = df.drop(range(0,3))
df = df.drop(range(163,166))
df = df.set_index('id')
df['description'] = df['description'].str.replace('Shop with Affiliates ', '')
df['description'] = df['description'].str.replace(' Mercury', '')

# Display table
df

Store the data to a CSV for later use

In [ ]:
# Outputs the DataFrame to a CSV
location = 'assets/pgcs_number_output.csv'
df.to_csv(location)

print('See assets folder for output')

### Generate URLs from DataFrame

In [ ]:
urls = []
fixed_url = 'https://www.pcgs.com/coinfacts/coin/'

for id in df.index:
  urls.append(fixed_url + id)

urls